Open this notebook in Google Colab : [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Riminder/hrflow-cookbook/blob/main/examples/%5BGrading%5D%20fast_grade_profiles_for_new_job.ipynb)

##### © 2025 HrFlow.ai — AI Research Department  
Licensed under the Apache License, Version 2.0.

In [ ]:
# Copyright 2025 HrFlow's AI Research Department. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

**Welcome to this Google Colaboratory tutorial for developers!**
This notebook is designed to help you to find similar candidates instantly based on **profile-to-profile matching**.

Here’s a quick overview of the notebook's workflow:

1. **[ ⚙️ Getting Started](#setup):** Install dependencies and authenticate with your HrFlow.ai API.
2. **[ 📤 Upload Resume](#upload-resume):** Upload a resume file from your local system.
3. **[ 📝 Parse a Resume in a Source](#resume-parsing):** Parse a Resume and create a Profile from it using the Parsing API.
4. **[ 👷 Profile Similarity](#matching):** Find Profiles similar to a target Profile.

<a name="setup"></a>
# **1. ⚙️ Getting Started:**
Let’s install and import the necessary Python packages to run the pipeline.

In [ ]:
!pip install --quiet hrflow

In [ ]:
from getpass import getpass
from hrflow import Hrflow

Authenticate with your HrFlow.ai API credentials

You’ll be prompted to enter:

- `API_SECRET`: Your HrFlow Read/Write API Secret Key →  
  Get it here: `https://{subdomain}.hrflow.ai/settings/api/keys`
  
- `API_USER`: Your HrFlow user email address


In [ ]:
API_SECRET = getpass("YOUR_API_SECRET")
API_USER = getpass("USER@EMAIL.DOMAIN")

client = Hrflow(api_secret=API_SECRET, api_user=API_USER)

<a name="upload-resume"></a>
# **2. 📤 Upload Resume:**

In [ ]:
from google.colab import files
binary_cv = list(files.upload().values())[0]

<a name="resume-parsing"></a>
# **3. 📝 Parse a Resume in a Hrflow.ai Source:**

Please fill the following variables with your own information:

`SOURCE_KEY`: The key (identifier) of the Source to which the Profile will be added.

In [ ]:
SOURCE_KEY = "xxx"

In [ ]:
profile_parsing = client.profile.parsing.add_file(SOURCE_KEY, profile_file=binary_cv, reference='resume_parsing')

<a name="matching"></a>
# **4. 👷 Profile Similarity:**

Please fill the following variables with your own information:

`SIMILARITY_SOURCE_KEYS` : The list of the keys of the Sources containing the targeted Profiles. Example : ["xxx", "yyy", "zzz"]

In [ ]:
SIMILARITY_SOURCE_KEYS = ["aaa", "bbb", "ccc"]

In [ ]:
def profile_matching(profile_key, source_key, source_keys):
    import requests
    import json

    url = "https://api.hrflow.ai/v1/profiles/matching"

    params = {
        "profile_key": profile_key,
        "source_key": source_key,
        "source_keys": json.dumps(source_keys),
        "page": 1,
        "limit": 300,
        "order_by": "desc",
    }

    headers = {
        "accept": "application/json",
        "X-API-KEY": API_SECRET,
        "X-USER-EMAIL": API_USER,
    }

    response = requests.get(url, params=params, headers=headers)
    return response.json()

Make API request to HrFlow.ai's matching endpoint to find similar profiles to a target candidate.

In [ ]:
PROFILE_KEY = profile_parsing['data']['profile']['key']

response = profile_matching(PROFILE_KEY, SOURCE_KEY, SIMILARITY_SOURCE_KEYS)

if response["code"] == 200:
    print("🎊 \033[92mCongrats !\033[0m You have successfully found profiles matching the target profile")
    print(response)
else :
    print("🚨 \033[91mSomething went wrong !\033[0m")
    print(response)

 Display the Top Scored Candidates with their matching scores grouped by source

In [ ]:
scores = [score_couple[1] for score_couple in response["data"]["predictions"]]

# Group profiles by source_key
source_profiles = {}
for profile, score in zip(response["data"]["profiles"], scores):
    source_key = profile["source"]["key"]
    if source_key not in source_profiles:
        source_profiles[source_key] = []
    source_profiles[source_key].append((profile, score))

# Print formatted results by source
for source_key, profiles in source_profiles.items():
    # Calculate average score for this source
    avg_score = sum(score for _, score in profiles) / len(profiles)
    print(f"# Source {source_key} (average score: \033[1m\033[92m{avg_score:.3f}\033[0m):")

    # Print each profile in this source
    for profile, score in profiles:
        profile_key = profile["key"]
        name = profile["info"]["full_name"]
        print(f"- profile \033[1m{profile_key}\033[0m {name}: \033[1m\033[92m{score:.3f}\033[0m")
    print()